In [19]:
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/red_pill_analises/'

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import re

df_channels = pd.read_csv(folder_path + 'data_base/dados_brutos/canais_brutos.csv')
df_videos = pd.read_csv(folder_path + 'data_base/dados_brutos/videos_brutos.csv')
df_comments = pd.read_csv(folder_path + 'data_base/dados_brutos/comentarios_brutos.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
# Mantenho apenas os vídeos que pertencem aos X canais
channels_validos = set(df_channels['channel_id'])
df_videos = df_videos[df_videos['channel_id'].isin(channels_validos)].copy()

# Garante que a coluna 'published_at' é do tipo datetime (timezone-aware)
df_videos['published_at'] = pd.to_datetime(df_videos['published_at'], utc=True)

# Filtra os vídeos após (ou em) 1º de janeiro de 2024 (com timezone UTC)
df_videos = df_videos[df_videos['published_at'] >= pd.Timestamp('2024-01-01', tz='UTC')]

# Mantenho apenas os comentários que pertencem aos vídeos dos X canais
videos_validos = set(df_videos['video_id'])
df_comments = df_comments[df_comments['video_id'].isin(videos_validos)].copy()
df_comments.to_csv(folder_path + 'data_base/dados_brutos/comentarios_brutos.csv')

In [21]:
df_channels.shape[0], df_videos.shape[0], df_comments.shape[0]

(28, 18034, 3037221)

In [22]:
df = pd.read_csv(folder_path + 'data_base/dados_brutos/comentarios_brutos.csv')
df.shape[0]

3037221

In [23]:
# Para garantir que a coleta foi feita corretamente
df_videos.groupby('channel_id').size().reset_index().get('channel_id').isin(df_channels.get('channel_id')).all()

np.True_

In [24]:
import pandas as pd
import re

# --- 2. ANONIMIZAÇÃO DOS CANAIS ---

# Contar vídeos por canal e ordenar do maior para o menor
channel_video_counts = df_videos['channel_id'].value_counts().reset_index()
channel_video_counts.columns = ['channel_id', 'video_count']

# Ordenar os canais pela contagem de vídeos (decrescente)
channel_video_counts = channel_video_counts.sort_values(by='video_count', ascending=False)

# Gerar o id_canal_anonimizado (canal_1, canal_2, ...)
channel_video_counts['id_canal_anonimizado'] = [f'canal_{i+1}' for i in range(len(channel_video_counts))]

# Juntar essa informação de volta no df_channels
df_channels = pd.merge(df_channels, channel_video_counts[['channel_id', 'id_canal_anonimizado']], on='channel_id')


In [25]:

df_videos = pd.merge(
    df_videos,
    df_channels[['channel_id', 'id_canal_anonimizado']], # Use o seu df_videos aqui
    on='channel_id',
    how='left'
)


In [26]:
# ==============================================================================
# --- 3. NOVA ANONIMIZAÇÃO DOS VÍDEOS (SEQUENCIAL) ---
# Este bloco substitui o seu código anterior.
# ==============================================================================
print("Iniciando a nova anonimização sequencial dos vídeos...")

df_videos_ordenado = df_videos.sort_values(by='view_count', ascending=False)
df_videos_ordenado = df_videos_ordenado.reset_index(drop=True)

# 3. Criar o 'id_video_anonimizado' usando o novo index sequencial
#    Usamos o `.index` do dataframe ordenado e somamos 1 para começar em 'video_1'
df_videos_ordenado['id_video_anonimizado'] = df_videos_ordenado.index.map(lambda i: f"video_{i + 1}")

# 4. (Opcional) Você pode remover a coluna auxiliar se não for mais necessária
#    E atribuir de volta para a variável df_videos, se desejar.
df_videos = df_videos_ordenado

print("Anonimização sequencial concluída com sucesso.")


Iniciando a nova anonimização sequencial dos vídeos...
Anonimização sequencial concluída com sucesso.


In [27]:
# Se quiser manter os colchetes no texto final, adicione no re.sub():
def anonimizar_texto_avancado(texto):
    if not isinstance(texto, str):
        return ''

    # Substituir menções e emails primeiro
    texto_anon = re.sub(r'@\w+', '[MENCAO]', texto)
    texto_anon = re.sub(r'\S+@\S+', '[EMAIL]', texto_anon)

    return texto_anon.strip()


In [28]:
# --- 5. APLICAÇÃO DA ANONIMIZAÇÃO NOS TEXTOS ---


df_channels['description'] = df_channels['description'].apply(anonimizar_texto_avancado)

colunas_texto = ['title', 'description']
# Aplicar em df_videos
for coluna in colunas_texto:
    if coluna in df_videos.columns:
        df_videos[coluna] = df_videos[coluna].apply(anonimizar_texto_avancado)



In [29]:
df_videos.columns

Index(['Unnamed: 0', 'video_id', 'title', 'description', 'channel_id',
       'published_at', 'category_id', 'tags', 'view_count', 'like_count',
       'comment_count', 'duration', 'definition', 'caption',
       'licensed_content', 'privacy_status', 'license', 'embeddable',
       'public_stats_viewable', 'is_made_for_kids', 'thumbnail_url',
       'default_audio_language', 'default_language', 'actual_start_time',
       'scheduled_start_time', 'actual_end_time', 'scheduled_end_time',
       'concurrent_viewers', 'active_live_chat_id', 'recording_date',
       'topicCategories', 'processing_status', 'parts_total',
       'parts_processed', 'time_left_ms', 'processing_failure_reason',
       'transcription', 'id_canal_anonimizado', 'id_video_anonimizado'],
      dtype='object')

In [30]:
df_comments.columns

Index(['Unnamed: 0.6', 'Unnamed: 0.5', 'Unnamed: 0.4', 'Unnamed: 0.3',
       'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'video_id', 'comment_id',
       'author', 'author_profile_image_url', 'author_channel_url',
       'author_channel_id', 'comment', 'published_at', 'updated_at',
       'like_count', 'viewer_rating', 'can_rate', 'is_reply', 'parent_id',
       'channel_id'],
      dtype='object')

In [31]:
import pandas as pd
import numpy as np # Usaremos para lidar com possíveis NaNs nos autores

# --------------------------------------------------------------------------
# --- INÍCIO DO NOVO CÓDIGO PARA ANONIMIZAR OS COMENTÁRIOS ---
# --------------------------------------------------------------------------

# --- Etapa 1: Juntar o id_video_anonimizado no df_comments ---
# Usamos 'how=left' para garantir que não perderemos comentários caso algum vídeo não seja encontrado.
# No nosso exemplo, troque df_videos por df_videos_anon_exemplo. No seu código final, use o df_videos completo.
df_comments = pd.merge(
    df_comments,
    df_videos[['video_id', 'id_video_anonimizado']], # Use o seu df_videos aqui
    on='video_id',
    how='left'
)


# --- Etapa 1.2: Juntar o id_canal_anonimizado no df_comments ---
# Usamos 'how=left' para garantir que não perderemos comentários caso algum vídeo não seja encontrado.
# No nosso exemplo, troque df_canals por df_videos_anon_exemplo. No seu código final, use o df_videos completo.
df_comments = pd.merge(
    df_comments,
    df_channels[['channel_id', 'id_canal_anonimizado']], # Use o seu df_videos aqui
    on='channel_id',
    how='left'
)



In [32]:
print(df_comments.columns)

Index(['Unnamed: 0.6', 'Unnamed: 0.5', 'Unnamed: 0.4', 'Unnamed: 0.3',
       'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'video_id', 'comment_id',
       'author', 'author_profile_image_url', 'author_channel_url',
       'author_channel_id', 'comment', 'published_at', 'updated_at',
       'like_count', 'viewer_rating', 'can_rate', 'is_reply', 'parent_id',
       'channel_id', 'id_video_anonimizado', 'id_canal_anonimizado'],
      dtype='object')


In [33]:
# --- Etapa 2: Anonimizar o Autor ---
# Pegar todos os autores únicos, removendo possíveis valores nulos (NaN)
autores_unicos = df_comments['author_channel_id'].dropna().unique()

# Criar um mapa de cada autor para um id anônimo (ex: 'Carlos' -> 'autor_1')
mapa_autores = {autor: f'autor_{i+1}' for i, autor in enumerate(autores_unicos)}

# Aplicar o mapa para criar a nova coluna.
# Comentários com autor nulo receberão o valor [autor_desconhecido].
df_comments['id_autor_anonimizado'] = df_comments['author_channel_id'].map(mapa_autores).fillna('[autor_desconhecido]')




In [34]:
# --- Etapa 3: Anonimizar o ID do Comentário ---
# Ordenar por vídeo e data para garantir a sequência correta dos comentários
df_comments = df_comments.sort_values(by=['id_video_anonimizado', 'published_at'])

# Criar o ID anônimo final do comentário
# Usamos .str.replace para extrair os números do id_video (ex: 'video_1_1' -> '1_1')
df_comments['id_comentario_anonimizado'] = df_comments.index.map(lambda i: f"comentario_{i + 1}")

# --- Etapa 4: Anonimizar o texto do comentário ---
# Reutilizamos a mesma função e mapa de canais que criamos anteriormente
if 'comment' in df_comments.columns:
    df_comments['comment'] = df_comments['comment'].apply(
        anonimizar_texto_avancado)


In [35]:
# Conte quantas vezes um ID de vídeo aparece mais de uma vez
num_duplicatas = df_videos['id_video_anonimizado'].duplicated().sum()

print(f"Existem {num_duplicatas} IDs de vídeo duplicados.")

Existem 0 IDs de vídeo duplicados.


In [36]:
(df_channels.drop(columns =['title', 'channel_id'])).to_csv(folder_path + 'data_base/dados_anonimizados/canais_anonimizados.csv', index=False)
(df_videos.drop(columns =['video_id','channel_id'])).to_csv(folder_path + 'data_base/dados_anonimizados/videos_anonimizados.csv', index=False)
(df_comments.drop(columns =['video_id','channel_id','comment_id','author',
       'author_profile_image_url', 'author_channel_url', 'author_channel_id'])).to_csv(folder_path + 'data_base/dados_anonimizados/comentarios_anonimizados.csv', index=False)